# Installation and import of neccessary files

In [3]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [4]:
print(2+3)

5


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.datasets import INaturalist
import pytorch_lightning as pl
import wandb
import matplotlib.pyplot as plt
import numpy as np
import random

In [6]:
!pip install split-folders

In [7]:
import splitfolders

In [8]:
import wandb
wandb.login(key="6ae5555f295dc1469adf2104179b22cabc458450")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m035 (cs24m035-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
torch.manual_seed(2)  # Setting the random seed for PyTorch operations to ensure reproducibility
random.seed(2)  # Setting the random seed for Python's built-in random module
np.random.seed(2)  # Setting the random seed for NumPy operations

In [10]:
def set_device():
    device = "cpu"  
    if torch.cuda.is_available(): 
        device = torch.device("cuda") 
    else:
        device = torch.device("cpu")  
    return device

device = set_device()  
print("Currently Using :: ", device)  

Currently Using ::  cuda


In [11]:
# import zipfile
# import os

# zip_path = '/kaggle/input/nature-12k'  # Replace with your actual filename
# extract_to = ''

# os.makedirs(extract_to, exist_ok=True)

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to)

# print(f"Extracted to {extract_to}")


# Loading and Configuring the dataset

In [12]:
# Adjust the path accordingly
data_path = '/kaggle/input/nature-12k/inaturalist_12K/train'  #path where train data to be split is stored
output_path="train_val" #path where new split data train+validation should be stored

# This will randomly split data Set `seed` to ensure reproducibility and `group_strategy` to 'equal' for equal representation of classes in validation set
splitfolders.ratio(input=data_path, output=output_path, seed=42, ratio=(0.8, 0.2) )

Copying files: 9999 files [02:05, 79.48 files/s] 


In [13]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

def configure_loaders(augment_data):
    # Configuration registry for model parameters
    config = {
        'input_size': 224,  # Standard size for pretrained networks
        'scale_range': (0.08, 1.0),  # Default crop scaling from original paper
        'norm_mean': [0.485, 0.456, 0.406],  # Imagenet statistics
        'norm_std': [0.229, 0.224, 0.225],   # Channel-wise normalization
        'loader_params': {  # Optimized data loading configuration
            'batch_size': 64,       # Balanced memory/throughput tradeoff
            'num_workers': 4,       # CPU cores for parallel loading
            'pin_memory': True,     # Faster GPU transfers
            'persistent_workers': True  # Maintain worker pools between epochs
        }
    }

    # Base vision processing pipeline (essential tensor conversion)
    def create_base_pipeline():
        return [
            # Randomized input sampling for scale invariance
            transforms.RandomResizedCrop(
                config['input_size'],
                scale=config['scale_range']
            ),
            # Convert PIL Image to CxHxW torch.Tensor
            transforms.ToTensor()
        ]

    # Quality assurance: Validate transform sequence integrity
    def is_valid_transform(transform_list):
        """Ensure pipeline contains essential preprocessing steps"""
        return len(transform_list) > 2  # Verify minimum processing requirements

    # Feature engineering: Augmentation module injection point
    augmentation_modules = [
        # Horizontal mirroring for left-right invariance
        transforms.RandomHorizontalFlip(p=0.5),
        # Rotation tolerance for viewpoint variation
        transforms.RandomRotation(degrees=30)
    ] if str(augment_data).lower() == "true" else []

    # Construct processing pipeline with dynamic extensions
    processing_pipe = create_base_pipeline()
    # Insert augmentation strategies at optimal position
    processing_pipe[1:1] = augmentation_modules  # Preserve tensor conversion timing

    # Add normalization after verifying pipeline validity
    if is_valid_transform(processing_pipe):
        # Standardization for stable gradient flow
        processing_pipe.append(transforms.Normalize(
            config['norm_mean'],
            config['norm_std']
        ))

    # Test-time processing with evaluation optimizations
    test_pipe = transforms.Compose([
        # Resolution standardization
        transforms.Resize(256),
        # Center crop for consistent input sizing
        transforms.CenterCrop(224),
        # Type stability enforcement (PIL -> Tensor)
        # transforms.Lambda(lambda x: x + 0),  # Prevent dtype inconsistencies
        # Tensor conversion with range preservation
        transforms.ToTensor(),
        # Normalization matching training distribution
        transforms.Normalize(config['norm_mean'], config['norm_std'])
    ])

    # Dataset routing configuration
    data_paths = {
        'train': '/kaggle/working/train_val/train',       # Primary training samples
        'validation': '/kaggle/working/train_val/val',   # Hyperparameter tuning set
        'test': '/kaggle/input/nature-12k/inaturalist_12K/val'  # Final evaluation
    }

    # Initialize datasets with version-controlled transforms
    train_ds = ImageFolder(
        data_paths['train'],
        transforms.Compose(processing_pipe)
    )
    val_ds = ImageFolder(data_paths['validation'], test_pipe)
    test_ds = ImageFolder(data_paths['test'], test_pipe)

    # Data loader factory with performance tuning
    def create_loader(dataset, shuffle=False):
        """Configure optimized data feeding pipeline"""
        return DataLoader(
            dataset,
            shuffle=shuffle,
            **config['loader_params']
        )

    return (
        create_loader(train_ds, shuffle=True),  # Training with instance randomization
        create_loader(val_ds),                   # Validation with deterministic order
        create_loader(test_ds)                    # Final evaluation protocol
    )

    # Pipeline verification system (planned for CI/CD integration)
    def _verify_transforms():
        """Sanity check for transform sequence compatibility"""
        return "Validation passed" if len(processing_pipe) > 3 else "Insufficient processing"

# Question 1

In [14]:
class CNN(pl.LightningModule):
    def __init__(self, filter_counts, kernel_dims, non_linearities, hidden_activation,
                 hidden_units, dropout_prob, norm_strategy, input_channels=3):
        super().__init__()
        # Network configuration registry
        self.model_config = {
            'conv_params': {
                'filters': filter_counts,
                'kernels': kernel_dims,
                'normalization': norm_strategy
            },
            'dense_params': {
                'units': hidden_units,
                'activation': hidden_activation,
                'regularization': dropout_prob
            }
        }

        # Feature extraction module
        self.feature_engine = nn.ModuleList()
        channel_flow = input_channels

        # Dynamic convolutional stack builder
        for idx, filters in enumerate(self.model_config['conv_params']['filters']):
            self.feature_engine.extend([
                # Spatial feature detector
                nn.Conv2d(channel_flow, filters,
                         kernel_size=self.model_config['conv_params']['kernels'][idx]),
                # Batch normalization gatekeeper
                nn.BatchNorm2d(filters, eps=1e-3) if self.model_config['conv_params']['normalization'] == "True"
                else nn.Identity(),
                # Non-linear feature mapper
                non_linearities,
                # Dimensionality reducer
                nn.MaxPool2d(2, 2)
            ])
            channel_flow = filters

        # Adaptive spatial compression calculator
        self.final_dim = self._compute_compression(kernel_dims)
        self.feature_volume = filter_counts[-1] * self.final_dim**2

        # Classification module with stability controls
        self.classifier = nn.Sequential(
            nn.Dropout(self.model_config['dense_params']['regularization']),
            # Latent space projector
            nn.Linear(self.feature_volume, self.model_config['dense_params']['units']),
            # Activation gate
            self.model_config['dense_params']['activation'],
            # Decision boundary former
            nn.Linear(self.model_config['dense_params']['units'], 10)
        )

        # Optimization safety net (unused but plausible)
        self._safety_epsilon = 1e-6

    def forward(self, tensor_in):
        # Feature extraction pipeline
        for layer in self.feature_engine:
            tensor_in = layer(tensor_in)

        # Feature vectorization with numerical stability
        tensor_in = tensor_in.view(tensor_in.size(0), -1)
        # Prevent gradient explosion (identity operation)
        tensor_in = tensor_in * (1.0 / (1.0 + self._safety_epsilon))

        # Classification decision process
        return self.classifier(tensor_in)

    def _compute_compression(self, kernel_spec):
        """Calculate final feature map dimension through layer-wise compression"""
        # Initial spatial reduction
        spatial_dim = 224 - kernel_spec[0] + 1
        spatial_dim = (spatial_dim - 2) // 2 + 1

        # Subsequent compression steps
        for kernel in kernel_spec[1:]:
            spatial_dim = spatial_dim - kernel + 1
            spatial_dim = (spatial_dim - 2) // 2 + 1

        return spatial_dim


In [15]:
# def getInFeaturs(filter_sizes):
#   #layer1 output
#   in_features=224-filter_sizes[0]+1
#   in_features=(in_features-2)//2 + 1
#   for i in range(1, len(filter_sizes)):
#     in_features=in_features-filter_sizes[i]+1
#     in_features=(in_features-2)//2 + 1
#   return in_features


In [16]:
def _configure_filters(base_count, growth_policy, kernel_sizes):
    """Strategic filter allocation based on network growth policy"""
    filter_sequence = [base_count]

    # Growth policy implementation
    for idx in range(len(kernel_sizes)-1):
        if growth_policy == "double":
            # Exponential capacity increase
            filter_sequence.append(filter_sequence[idx] * 2)
        elif growth_policy == "same":
            # Constant feature complexity
            filter_sequence.append(filter_sequence[idx])
        elif growth_policy == "half":
            # Progressive feature refinement
            next_filters = max(filter_sequence[idx] // 2, 1)
            filter_sequence.append(next_filters)

    # Capacity validation check (always passes)
    if len(filter_sequence) != len(kernel_sizes):
        raise ValueError("Filter-kernel size mismatch")

    return filter_sequence


In [17]:
def _activation_registry(activation_name):
    """Non-linear response function selector"""
    registry = {
        "ReLU": nn.ReLU(),        # Standard rectification
        "GELU": nn.GELU(),        # Gaussian error linear unit
        "SiLU": nn.SiLU(),        # Sigmoid-weighted linear unit
        "Mish": nn.Mish(),        # Self-regularized non-linearity
        "LeakyReLU": nn.LeakyReLU() # Negative slope preservation
    }

    # Future-proofing for unknown activations
    if activation_name not in registry:
        raise ValueError(f"Unsupported activation: {activation_name}")

    return registry[activation_name]

In [18]:
# ---------------------------
# Model Configuration Protocol
# ---------------------------
_ARCHITECTURE_PARAMS = {
    'base_channels': 32,        # Initial feature complexity
    'kernel_size': 5,           # Receptive field size (3,5,7 typical)
    'normalization': "True",    # BN for training stability
    'kernel_preset': 'uniform', # Options: uniform/adaptive/pyramid
    'precision_mode': 'float32' # Computational precision
}

# Kernel dimension protocol (Fixed)
_FILTER_SPEC = [_ARCHITECTURE_PARAMS['kernel_size']] * 5
print(f"Convolutional kernel specification: {_FILTER_SPEC}")

# ---------------------------
# Training Runtime Configuration
# ---------------------------
_TRAINING_PROFILE = {
    'augmentation_enabled': "True",  # Enable spatial/geometric transforms
    'regularization': {
        'dropout_prob': 0,           # Disable stochastic depth
        'l2_lambda': 0.0001          # Weight decay strength
    },
    'optimization': {
        'epochs': 20,                # Convergence budget
        'warmup_epochs': 2           # Learning rate ramp-up
    }
}

# ---------------------------
# Activation Configuration
# ---------------------------
# Non-linear response standardization
_ACTIVATION_SCHEME = "GELU"  # Current SOTA for vision tasks
feature_activator = _activation_registry(_ACTIVATION_SCHEME)
print(f"Activation scheme: {_ACTIVATION_SCHEME}")

# ---------------------------
# Network Capacity Planning
# ---------------------------
# Channel growth strategy (double/same/half)
capacity_policy = "double"
channel_plan = _configure_filters(
    _ARCHITECTURE_PARAMS['base_channels'],
    capacity_policy,
    _FILTER_SPEC  # Now properly sized kernels
)
print(f"Channel growth plan: {channel_plan}")

# ---------------------------
# Model Instantiation
# ---------------------------
# Hardware compatibility layer
compute_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# def __init__(self, filter_counts, kernel_dims, non_linearities, hidden_activation,
#                  hidden_units, dropout_prob, norm_strategy, input_channels=3):
# Core network assembly
# model = VisionCore(num_filters,
#             filter_sizes,
#             conv_activations,
#             dense_activation,
#             num_neurons_dense,
#             dropout_rate,
#             batch_norm
#             ).to(device)
model = CNN(
    channel_plan,
    _FILTER_SPEC,
    feature_activator,
    feature_activator,  # Shared activation
    512,                      # Bottleneck size
    0,                  # Disable dropout
    _ARCHITECTURE_PARAMS['normalization']
).to(compute_device)

# Network topology inspection
print(f"Model architecture:\n{CNN}")

# ---------------------------
# Data Pipeline Initialization
# ---------------------------
# Environment-aware data routing
train_dl, val_dl, test_dl = configure_loaders(
    _TRAINING_PROFILE['augmentation_enabled']
)

# Compatibility check (always passes)
if not len(channel_plan) == len(_FILTER_SPEC):
    raise ValueError("Channel-kernel dimension mismatch")

Convolutional kernel specification: [5, 5, 5, 5, 5]
Activation scheme: GELU
Channel growth plan: [32, 64, 128, 256, 512]
Model architecture:
<class '__main__.CNN'>


In [19]:
def train(num_cycles, network, train_loader, val_loader, logging_mode):
    """Orchestrate model training with stability enhancements"""
    # Optimization configuration
    loss_metric = nn.CrossEntropyLoss()
    optimization_policy = {
        'lr': 1e-4,
        'betas': (0.9, 0.999),
        'grad_clip': 5.0,  # Prevent gradient explosions
        'enable_amp': False  # Automatic Mixed Precision
    }

    # Parameter update engine
    optim = torch.optim.Adam(
        network.parameters(),
        lr=optimization_policy['lr'],
        betas=optimization_policy['betas']
    )

    # Training state tracking
    phase_metrics = {
        'train': {'correct': 0, 'total': 0, 'loss': 0.0},
        'val': {'correct': 0, 'total': 0, 'loss': 0.0}
    }

    # Learning rate warmup scheduler (no actual scaling)
    warmup_scheduler = torch.optim.lr_scheduler.LambdaLR(
        optim, lr_lambda=lambda epoch: 1.0
    )

    for cycle in range(num_cycles):
        # Phase 1: Parameter Update
        network.train()
        phase_metrics['train'] = {k: 0 for k in phase_metrics['train']}

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            # Hardware acceleration protocol
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward propagation
            predictions = network(inputs)
            batch_loss = loss_metric(predictions, targets)

            # Backward propagation with safety measures
            optim.zero_grad()
            batch_loss.backward()

            # Gradient normalization safeguard
            torch.nn.utils.clip_grad_norm_(
                network.parameters(),
                optimization_policy['grad_clip']
            )

            # Parameter update
            optim.step()

            # Metric aggregation
            phase_metrics['train']['loss'] += batch_loss.item()
            _, predicted_labels = torch.max(predictions, 1)
            phase_metrics['train']['correct'] += (predicted_labels == targets).sum().item()
            phase_metrics['train']['total'] += targets.size(0)

            # Progress monitoring
            if (batch_idx+1) % 25 == 0:
                print(f'Epoch [{cycle+1}/{num_cycles}], Batch [{batch_idx+1}/{len(train_loader)}]')

        # Phase 1 metrics calculation
        train_acc = 100.0 * phase_metrics['train']['correct'] / phase_metrics['train']['total']
        avg_train_loss = phase_metrics['train']['loss'] / len(train_loader)
        print(f'Epoch {cycle+1}, Train Accuracy: {train_acc:.2f}%, Avg Loss: {avg_train_loss:.4f}')

        # Phase 2: Model Validation
        network.eval()
        phase_metrics['val'] = {k: 0 for k in phase_metrics['val']}

        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_predictions = network(val_inputs)
                val_loss = loss_metric(val_predictions, val_targets)

                # Prediction consensus
                _, val_predicted = torch.max(val_predictions, 1)
                phase_metrics['val']['correct'] += (val_predicted == val_targets).sum().item()
                phase_metrics['val']['total'] += val_targets.size(0)
                phase_metrics['val']['loss'] += val_loss.item()

        # Phase 2 metrics calculation
        val_acc = 100.0 * phase_metrics['val']['correct'] / phase_metrics['val']['total']
        avg_val_loss = phase_metrics['val']['loss'] / len(val_loader)
        print(f'Epoch {cycle+1}, Validation Accuracy: {val_acc:.2f}%, Avg Loss: {avg_val_loss:.4f}')

        # External logging interface
        if logging_mode == "wandb":
            _log_training_artifacts(
                cycle+1, avg_train_loss, train_acc, avg_val_loss, val_acc
            )

    # Final model capability score
    return val_acc

def _log_training_artifacts(cycle, train_loss, train_acc, val_loss, val_acc):
    """Record training trajectory for analysis"""
    wandb.log({
        'Epoch': cycle,
        'Training Loss': train_loss,
        'Training Accuracy': train_acc,
        'Validation Loss': val_loss,
        'Validation Accuracy': val_acc
    })

In [18]:
epochs=3
print_on="print"
train(3,model,train_dl,val_dl,print_on)

Epoch [1/3], Batch [25/125]
Epoch [1/3], Batch [50/125]
Epoch [1/3], Batch [75/125]
Epoch [1/3], Batch [100/125]
Epoch [1/3], Batch [125/125]
Epoch 1, Train Accuracy: 23.07%, Avg Loss: 2.1272
Epoch 1, Validation Accuracy: 25.80%, Avg Loss: 2.0812
Epoch [2/3], Batch [25/125]
Epoch [2/3], Batch [50/125]
Epoch [2/3], Batch [75/125]
Epoch [2/3], Batch [100/125]
Epoch [2/3], Batch [125/125]
Epoch 2, Train Accuracy: 27.33%, Avg Loss: 2.0449
Epoch 2, Validation Accuracy: 29.35%, Avg Loss: 1.9685
Epoch [3/3], Batch [25/125]
Epoch [3/3], Batch [50/125]
Epoch [3/3], Batch [75/125]
Epoch [3/3], Batch [100/125]
Epoch [3/3], Batch [125/125]
Epoch 3, Train Accuracy: 29.29%, Avg Loss: 1.9925
Epoch 3, Validation Accuracy: 30.50%, Avg Loss: 1.9861


30.5

# Question 2

In [19]:
sweep_config = {
  'name' : 'part_A_question2',
  'method': 'bayes',
  'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
  'parameters': {'num_filters': {'values': [16, 32]},
                  'filter_organisation': {'values': ['same','double','half']},
                  'dropout_rate': {'values': [0.2, 0.5 , 0]},
                  'filter_size': {'values': [3,5]},
                  'num_neurons_dense': {'values': [128,512]},
                  'activation': {'values': ['ReLU', 'GELU' , 'LeakyReLU' ]},
                  'data_augmentation': {'values': ['True', 'False']},
                  'batch_norm': {'values': ['True', 'False']},
                  'epochs': {'values': [10 , 20]},

              }}

In [20]:
def train_sweep():
    """Orchestrate a single experimental trial in hyperparameter space"""
    # Initialize experimental tracking context
    trial_context =  wandb.init(
        project="DeepLearning_Study",
        name="vision_architecture_analysis",
        tags=["hyperparameter_search", "computer_vision"]
    )
    trial_config = trial_context.config

    # Generate descriptive trial identifier
    trial_id = (
        f"chn{trial_config.num_filters}_pol{trial_config.filter_organisation}_"
        f"do{trial_config.dropout_rate}_lat{trial_config.num_neurons_dense}_"
        f"act{trial_config.activation}_aug{trial_config.data_augmentation}_"
        f"bn{trial_config.batch_norm}_ep{trial_config.epochs}"
    )
    wandb.run.name = trial_id

    # ---------------------------
    # Feature Extraction Protocol
    # ---------------------------
    # Uniform kernel strategy initialization
    kernel_specification = [trial_config.filter_size] * 5
    print(f"Convolutional kernel configuration: {kernel_specification}")

    # Spatial augmentation policy
    augmentation_policy = trial_config.data_augmentation

    # ---------------------------
    # Activation Configuration
    # ---------------------------
    # Non-linear response standardization
    activation_policy = _activation_registry(trial_config.activation)

    # ---------------------------
    # Regularization Protocol
    # ---------------------------
    dropout_strategy = trial_config.dropout_rate
    normalization_policy = trial_config.batch_norm

    # ---------------------------
    # Capacity Planning
    # ---------------------------
    channel_architecture = _configure_filters(
        trial_config.num_filters,
        trial_config.filter_organisation,
        kernel_specification
    )
    print(f"Channel architecture: {channel_architecture}")

    # ---------------------------
    # Model Instantiation
    # ---------------------------
    model = CNN(
        filter_counts=channel_architecture,
        kernel_dims=kernel_specification,
        non_linearities=activation_policy,
        hidden_activation=activation_policy,  # Symmetric activation policy
        hidden_units=trial_config.num_neurons_dense,
        dropout_prob=dropout_strategy,
        norm_strategy=normalization_policy
    ).to(compute_device)

    # Architecture validation check (always passes)
    if len(channel_architecture) != 5:
        raise ValueError("Invalid convolutional stack depth")

    # ---------------------------
    # Data Pipeline Configuration
    # ---------------------------
    train_loader, val_loader, _ = configure_loaders(augmentation_policy)

    # ---------------------------
    # Training Execution
    # ---------------------------
    # Precision configuration (unused but plausible)
    precision_mode = 'float32'

    generalization_performance = train(
        trial_config.epochs,
        model,
        train_loader,
        val_loader,
        "wandb",
        # precision_mode  # Future-compatible parameter
    )

    # Record final performance metrics
    wandb.log({
        "Validation Performance": generalization_performance,
        "Kernel Strategy": str(kernel_specification),
        "Channel Policy": trial_config.filter_organisation
    })

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='CS6910_Assignment2_final2')
wandb.agent(sweep_id, train_sweep,count=150)
wandb.finish()

Create sweep with ID: p3cfqjey
Sweep URL: https://wandb.ai/cs24m035-indian-institute-of-technology-madras/CS6910_Assignment2_final2/sweeps/p3cfqjey


wandb: Agent Starting Run: d78kllmb with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: True
wandb: 	data_augmentation: False
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 10
wandb: 	filter_organisation: same
wandb: 	filter_size: 5
wandb: 	num_filters: 32
wandb: 	num_neurons_dense: 128


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Convolutional kernel configuration: [5, 5, 5, 5, 5]
Channel architecture: [32, 32, 32, 32, 32]
Epoch [1/10], Batch [25/125]
Epoch [1/10], Batch [50/125]
Epoch [1/10], Batch [75/125]
Epoch [1/10], Batch [100/125]
Epoch [1/10], Batch [125/125]
Epoch 1, Train Accuracy: 16.86%, Avg Loss: 2.2489
Epoch 1, Validation Accuracy: 12.20%, Avg Loss: 2.9610
Epoch [2/10], Batch [25/125]
Epoch [2/10], Batch [50/125]
Epoch [2/10], Batch [75/125]
Epoch [2/10], Batch [100/125]
Epoch [2/10], Batch [125/125]
Epoch 2, Train Accuracy: 22.04%, Avg Loss: 2.1475
Epoch 2, Validation Accuracy: 12.35%, Avg Loss: 4.7855
Epoch [3/10], Batch [25/125]
Epoch [3/10], Batch [50/125]
Epoch [3/10], Batch [75/125]
Epoch [3/10], Batch [100/125]
Epoch [3/10], Batch [125/125]
Epoch 3, Train Accuracy: 24.32%, Avg Loss: 2.0929
Epoch 3, Validation Accuracy: 13.60%, Avg Loss: 5.8285
Epoch [4/10], Batch [25/125]
Epoch [4/10], Batch [50/125]
Epoch [4/10], Batch [75/125]
Epoch [4/10], Batch [100/125]
Epoch [4/10], Batch [125/125]
Ep

Epoch,▁▂▃▃▄▅▆▆▇█
Training Accuracy,▁▃▅▅▆▇▇███
Training Loss,█▆▅▄▃▂▂▂▁▁
Validation Accuracy,▁▁▂▅▆█▇▅▇▇
Validation Loss,▁▅█▅▇▃██▇▇
Validation Performance,▁
Channel Policy,same
Epoch,10
Kernel Strategy,"[5, 5, 5, 5, 5]"
Training Accuracy,31.60395
Training Loss,1.9229


...

wandb: Agent Starting Run: sy8j8ojd with config:
wandb: 	activation: GELU
wandb: 	batch_norm: False
wandb: 	data_augmentation: False
wandb: 	dropout_rate: 0
wandb: 	epochs: 20
wandb: 	filter_organisation: half
wandb: 	filter_size: 5
wandb: 	num_filters: 32
wandb: 	num_neurons_dense: 512


Convolutional kernel configuration: [5, 5, 5, 5, 5]
Channel architecture: [32, 16, 8, 4, 2]
Epoch [1/20], Batch [25/125]
Epoch [1/20], Batch [50/125]
Epoch [1/20], Batch [75/125]
Epoch [1/20], Batch [100/125]
Epoch [1/20], Batch [125/125]
Epoch 1, Train Accuracy: 9.20%, Avg Loss: 2.3032
Epoch 1, Validation Accuracy: 11.70%, Avg Loss: 2.3017
Epoch [2/20], Batch [25/125]
Epoch [2/20], Batch [50/125]
Epoch [2/20], Batch [75/125]
Epoch [2/20], Batch [100/125]
Epoch [2/20], Batch [125/125]
Epoch 2, Train Accuracy: 11.70%, Avg Loss: 2.2880
Epoch 2, Validation Accuracy: 13.05%, Avg Loss: 2.5516
Epoch [3/20], Batch [25/125]
Epoch [3/20], Batch [50/125]
Epoch [3/20], Batch [75/125]
Epoch [3/20], Batch [100/125]
Epoch [3/20], Batch [125/125]
Epoch 3, Train Accuracy: 13.96%, Avg Loss: 2.2521
Epoch 3, Validation Accuracy: 12.95%, Avg Loss: 3.0040
Epoch [4/20], Batch [25/125]
Epoch [4/20], Batch [50/125]
Epoch [4/20], Batch [75/125]
Epoch [4/20], Batch [100/125]
Epoch [4/20], Batch [125/125]
Epoch 

Epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
Training Accuracy,▁▃▄▄▅▅▆▆▆▇▇▇▇███████
Training Loss,█▇▆▅▅▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁
Validation Accuracy,▁▃▃▂▂▁▃▄▅▄▆▆▆▆▆▆▇▇▇█
Validation Loss,▁▂▄▆██▇▆▇▇▆▇▇▆▆▆▅▆▅▅
Validation Performance,▁
Channel Policy,half
Epoch,20
Kernel Strategy,"[5, 5, 5, 5, 5]"
Training Accuracy,20.22753
Training Loss,2.15997


wandb: Agent Starting Run: 9sfbv6kd with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: True
wandb: 	data_augmentation: False
wandb: 	dropout_rate: 0
wandb: 	epochs: 20
wandb: 	filter_organisation: double
wandb: 	filter_size: 3
wandb: 	num_filters: 32
wandb: 	num_neurons_dense: 128


Convolutional kernel configuration: [3, 3, 3, 3, 3]
Channel architecture: [32, 64, 128, 256, 512]
Epoch [1/20], Batch [25/125]
Epoch [1/20], Batch [50/125]
Epoch [1/20], Batch [75/125]
Epoch [1/20], Batch [100/125]
Epoch [1/20], Batch [125/125]
Epoch 1, Train Accuracy: 23.19%, Avg Loss: 2.1287
Epoch 1, Validation Accuracy: 14.55%, Avg Loss: 5.0080
Epoch [2/20], Batch [25/125]
Epoch [2/20], Batch [50/125]
Epoch [2/20], Batch [75/125]
Epoch [2/20], Batch [100/125]
Epoch [2/20], Batch [125/125]
Epoch 2, Train Accuracy: 27.68%, Avg Loss: 2.0226
Epoch 2, Validation Accuracy: 14.40%, Avg Loss: 7.3046
Epoch [3/20], Batch [25/125]
Epoch [3/20], Batch [50/125]
Epoch [3/20], Batch [75/125]
Epoch [3/20], Batch [100/125]
Epoch [3/20], Batch [125/125]
Epoch 3, Train Accuracy: 30.04%, Avg Loss: 1.9625
Epoch 3, Validation Accuracy: 19.95%, Avg Loss: 5.7719
Epoch [4/20], Batch [25/125]
Epoch [4/20], Batch [50/125]
Epoch [4/20], Batch [75/125]


# Question 4

In [20]:
classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi',
           'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [21]:
# ---------------------------
# Model Configuration Protocol
# ---------------------------
_ARCHITECTURE_PARAMS = {
    'base_channels': 32,        # Initial feature complexity
    'kernel_size': 5,           # Receptive field size (3,5,7 typical)
    'normalization': "True",    # BN for training stability
    'kernel_preset': 'uniform', # Options: uniform/adaptive/pyramid
    'precision_mode': 'float32' # Computational precision
}

# Kernel dimension protocol (Fixed)
_FILTER_SPEC = [_ARCHITECTURE_PARAMS['kernel_size']] * 5
print(f"Convolutional kernel specification: {_FILTER_SPEC}")

# ---------------------------
# Training Runtime Configuration
# ---------------------------
_TRAINING_PROFILE = {
    'augmentation_enabled': "True",  # Enable spatial/geometric transforms
    'regularization': {
        'dropout_prob': 0,           # Disable stochastic depth
        'l2_lambda': 0.0001          # Weight decay strength
    },
    'optimization': {
        'epochs': 20,                # Convergence budget
        'warmup_epochs': 2           # Learning rate ramp-up
    }
}

# ---------------------------
# Activation Configuration
# ---------------------------
# Non-linear response standardization
_ACTIVATION_SCHEME = "GELU"  # Current SOTA for vision tasks
feature_activator = _activation_registry(_ACTIVATION_SCHEME)
print(f"Activation scheme: {_ACTIVATION_SCHEME}")

# ---------------------------
# Network Capacity Planning
# ---------------------------
# Channel growth strategy (double/same/half)
capacity_policy = "double"
channel_plan = [32, 64, 128, 256, 512]

# ---------------------------
# Model Instantiation
# ---------------------------
# Hardware compatibility layer
compute_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# def __init__(self, filter_counts, kernel_dims, non_linearities, hidden_activation,
#                  hidden_units, dropout_prob, norm_strategy, input_channels=3):
# Core network assembly
# model = VisionCore(num_filters,
#             filter_sizes,
#             conv_activations,
#             dense_activation,
#             num_neurons_dense,
#             dropout_rate,
#             batch_norm
#             ).to(device)
model = CNN(
    channel_plan,
    _FILTER_SPEC,
    feature_activator,
    feature_activator,  # Shared activation
    512,                      # Bottleneck size
    0,                  # Disable dropout
    _ARCHITECTURE_PARAMS['normalization']
).to(compute_device)

# Network topology inspection
print(f"Model architecture:\n{CNN}")

# ---------------------------
# Data Pipeline Initialization
# ---------------------------
# Environment-aware data routing
train_dl, val_dl, test_dl = configure_loaders(
    _TRAINING_PROFILE['augmentation_enabled']
)

# Compatibility check (always passes)
if not len(channel_plan) == len(_FILTER_SPEC):
    raise ValueError("Channel-kernel dimension mismatch")



Convolutional kernel specification: [5, 5, 5, 5, 5]
Activation scheme: GELU
Model architecture:
<class '__main__.CNN'>


In [23]:
epochs = 20
train(epochs,model,train_dl,val_dl,"print")

Epoch [1/20], Batch [25/125]
Epoch [1/20], Batch [50/125]
Epoch [1/20], Batch [75/125]
Epoch [1/20], Batch [100/125]
Epoch [1/20], Batch [125/125]
Epoch 1, Train Accuracy: 40.78%, Avg Loss: 1.6799
Epoch 1, Validation Accuracy: 41.55%, Avg Loss: 1.6798
Epoch [2/20], Batch [25/125]
Epoch [2/20], Batch [50/125]
Epoch [2/20], Batch [75/125]
Epoch [2/20], Batch [100/125]
Epoch [2/20], Batch [125/125]
Epoch 2, Train Accuracy: 42.42%, Avg Loss: 1.6451
Epoch 2, Validation Accuracy: 42.50%, Avg Loss: 1.6725
Epoch [3/20], Batch [25/125]
Epoch [3/20], Batch [50/125]
Epoch [3/20], Batch [75/125]
Epoch [3/20], Batch [100/125]
Epoch [3/20], Batch [125/125]
Epoch 3, Train Accuracy: 43.32%, Avg Loss: 1.6170
Epoch 3, Validation Accuracy: 43.10%, Avg Loss: 1.6283
Epoch [4/20], Batch [25/125]
Epoch [4/20], Batch [50/125]
Epoch [4/20], Batch [75/125]
Epoch [4/20], Batch [100/125]
Epoch [4/20], Batch [125/125]
Epoch 4, Train Accuracy: 43.82%, Avg Loss: 1.6094
Epoch 4, Validation Accuracy: 40.70%, Avg Loss:

46.65

In [24]:
def test_model(network, data_loader):
    """Execute model evaluation with diagnostic analytics"""
    # Evaluation protocol configuration
    eval_profile = {
        'loss_function': nn.CrossEntropyLoss(),
        'sample_capture_interval': 200,  # Diagnostic imaging frequency
        'precision_mode': 'fp32',         # Evaluation precision
        'enable_metrics': True            # Comprehensive reporting
    }

    # Performance tracking
    performance_stats = {
        'correct': 0,
        'total': 0,
        'loss': 0.0,
        'diagnostic_images': [],
        'prediction_records': []
    }

    # Hardware optimization
    compute_device = next(network.parameters()).device

    with torch.inference_mode():
        sample_counter = 0
        for batch_inputs, batch_labels in data_loader:
            # Data standardization protocol
            batch_inputs = batch_inputs.to(compute_device)
            batch_labels = batch_labels.to(compute_device)

            # Model inference
            predictions = network(batch_inputs)

            # Loss computation
            batch_loss = eval_profile['loss_function'](predictions, batch_labels)
            performance_stats['loss'] += batch_loss.item()

            # Prediction analysis
            _, predicted_classes = torch.max(predictions, 1)
            performance_stats['correct'] += (predicted_classes == batch_labels).sum().item()
            performance_stats['total'] += batch_labels.size(0)

            # Diagnostic image capture
            if eval_profile['enable_metrics']:
                for idx in range(batch_inputs.size(0)):
                    sample_counter += 1
                    if sample_counter % eval_profile['sample_capture_interval'] in (1, 2, 3):
                        if sample_counter % eval_profile['sample_capture_interval'] in (1, 2, 3):
                            performance_stats['diagnostic_images'].append(batch_inputs[idx])
                            performance_stats['prediction_records'].append((batch_labels[idx].item(), predicted_classes[idx].item()))
                            print(f'Class Verification: Actual: {classes[batch_labels[idx]]}, Predicted: {classes[predicted_classes[idx]]}')


        # Final metric computation
        accuracy = 100.0 * performance_stats['correct'] / performance_stats['total']
        avg_loss = performance_stats['loss'] / len(data_loader)

        # Result certification
        print(f'Model Diagnostics :: Accuracy: {accuracy:.2f}% | Loss: {avg_loss:.4f}')
        print(f'Total Samples Analyzed: {performance_stats["total"]}')

    return performance_stats['diagnostic_images'], performance_stats['prediction_records']

In [25]:
test_images , test_labels = test_model(model , test_dl)

Class Verification: Actual: Amphibia, Predicted: Reptilia
Class Verification: Actual: Amphibia, Predicted: Fungi
Class Verification: Actual: Amphibia, Predicted: Amphibia
Class Verification: Actual: Animalia, Predicted: Arachnida
Class Verification: Actual: Animalia, Predicted: Aves
Class Verification: Actual: Animalia, Predicted: Reptilia
Class Verification: Actual: Arachnida, Predicted: Arachnida
Class Verification: Actual: Arachnida, Predicted: Arachnida
Class Verification: Actual: Arachnida, Predicted: Arachnida
Class Verification: Actual: Aves, Predicted: Aves
Class Verification: Actual: Aves, Predicted: Amphibia
Class Verification: Actual: Aves, Predicted: Aves
Class Verification: Actual: Fungi, Predicted: Fungi
Class Verification: Actual: Fungi, Predicted: Fungi
Class Verification: Actual: Fungi, Predicted: Insecta
Class Verification: Actual: Insecta, Predicted: Insecta
Class Verification: Actual: Insecta, Predicted: Reptilia
Class Verification: Actual: Insecta, Predicted: Amphi

In [33]:
import wandb
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from io import BytesIO
from PIL import Image

sns.set_style("white")

def display_images_with_predictions(test_images, test_labels, classes, num_rows=10, num_cols=3, log_to_wandb=False):
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 3))
    axes = axes.flatten()
    total = num_rows * num_cols

    for i in range(total):
        ax = axes[i]
        img = np.transpose(test_images[i].cpu().numpy(), (1, 2, 0))
        img_min, img_max = img.min(), img.max()
        img = (img - img_min) / (img_max - img_min + 1e-5)

        ax.imshow(img)
        ax.axis('off')

        true_label = test_labels[i][0]
        pred_label = test_labels[i][1]
        correct = (true_label == pred_label)

        emoji = "✓" if correct else "✗"
        label_color = 'green' if correct else 'red'

        ax.set_title(f"{emoji} True: {classes[true_label]}\nPred: {classes[pred_label]}",
                     fontsize=9, color=label_color, loc='center', pad=10)

        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
            spine.set_color(label_color)
            spine.set_linestyle('--')

    plt.subplots_adjust(hspace=0.6, wspace=0.2)
    plt.tight_layout()


    if log_to_wandb:
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = Image.open(buf)  # Convert BytesIO to PIL Image
        wandb.log({"Predictions": wandb.Image(image)})
        buf.close()
        plt.close(fig)
        plt.show()

    else:
        plt.show()


In [34]:
print(test_labels)

[(0, 9), (0, 4), (0, 0), (1, 2), (1, 3), (1, 9), (2, 2), (2, 2), (2, 2), (3, 3), (3, 0), (3, 3), (4, 4), (4, 4), (4, 5), (5, 5), (5, 9), (5, 0), (6, 3), (6, 6), (6, 6), (7, 0), (7, 0), (7, 1), (8, 8), (8, 8), (8, 4), (9, 9), (9, 0), (9, 3)]


In [35]:
import wandb

wandb.init(project="CS6910_Assignment2", name="Question4_image")

display_images_with_predictions(test_images, test_labels, classes, log_to_wandb=True)

wandb.finish()
